# Chat UI for Multimodal Article QA Assistant

Code authored by: Shaw Talebi

[Blog link](https://medium.com/towards-data-science/multimodal-rag-process-any-file-type-with-ai-e6921342c903) 
| [Video link](https://youtu.be/Y7pNmocrmi8)

modified by: Soonwook Hwang

### imports

In [1]:
#!pip install gradio

In [1]:
import ollama
from torch import load
import gradio as gr
import time
from functions import *

### Load model and data

In [2]:
# pull model
ollama.pull('llama3.2-vision')

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [3]:
# load article contents
text_content_list = load_from_json('data/text_content.json')
image_content_list = load_from_json('data/image_content.json')

# load embeddings
text_embeddings = load('data/text_embeddings.pt', weights_only=True)
image_embeddings = load('data/image_embeddings.pt', weights_only=True)

### UI

In [5]:
# Function to interact with the Ollama model
def stream_chat(message, history):
    """
    Streams the response from the Ollama model and sends it to the Gradio UI.
    
    Args:
        message (str): The user input message.
        history (list): A list of previous conversation messages.
        
    Yields:
        str: The chatbot's response chunk by chunk.
    """

    # context retrieval
    text_results, image_results = context_retrieval(message["text"], text_embeddings, image_embeddings, text_content_list, image_content_list)

    # construct prompt
    prompt = construct_prompt(message["text"], text_results, image_results)
    
    # Append the user message to the conversation history
    history.append({"role": "user", "content": prompt, "images": [image["image_path"] for image in image_results]})
    
    # Initialize streaming from Ollama
    stream = ollama.chat(
        model='llama3.2-vision',
        messages=history,  # Full chat history including the current user message
        stream=True,
    )
    
    response_text = ""
    for chunk in stream:
        content = chunk['message']['content']
        response_text += content
        yield response_text  # Send the response incrementally to the UI

    # Append the assistant's full response to the history
    history.append({"role": "assistant", "content": response_text})

In [6]:
gr.ChatInterface(
    fn=stream_chat,
    examples=[{"text": "What is CLIP's contrastive loss function?"}, 
              {"text": "What are the three paths described for making LLMs multimodal?"},
              {"text": "What is an intuitive explanation of multimodal embeddings?"}],
    multimodal=True,
).launch(
    server_name="0.0.0.0",
    server_port=7862,
    share=False  # Don't try to create public link
)

* Running on local URL:  http://0.0.0.0:7862
* To create a public link, set `share=True` in `launch()`.


In [7]:
!lsof -i :7862

COMMAND     PID   USER   FD   TYPE     DEVICE SIZE/OFF NODE NAME
python3.1 50158 qualis   82u  IPv4 1022583756      0t0  TCP *:7862 (LISTEN)


In [9]:
!hostname

gpu33


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


### Create SSH tunnel

ssh -L localhost:7862:gpu33:7862 qualis@neuron.ksc.re.kr